## 1. Farmers' Market Expansion
<p>You have been hired by a farming organisation that helps local farmers sell their products. They want to know whether they should open up a new farmers' market to sell dairy products from nearby farmers. They have supplied you with daily shopping data from a panel of local households from 2019 to 2020. </p>
<p>The organization will make their decision based on whether dairy products are popular in the area, and whether sales are trending in a positive direction. To answer these questions, they want three pieces of data:</p>
<ol>
<li>What was the total number of purchases of dairy products for each month of 2020 (i.e., the <code>total_sales</code>)?</li>
<li>What was the total share of dairy products (out of all products purchased) for each month of 2020 (i.e., the <code>market_share</code>)?</li>
<li>For each month of 2020, what was the percentage increase or decrease in total monthly dairy purchases compared to the same month in 2019 (i.e., the <code>year_change</code>)?</li>
</ol>
<p>The organization handles not only dairy farmers, but also those with chicken farms. As a result, they are only interested in these three categories (which they treat as dairy): ‘whole milk’, 'yogurt' and 'domestic eggs'.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5960/datasets/463543c8c38957ca5b95d93b02f2cb1bec53334f/diagram.PNG" alt="Database Schema" width="400px"></p>

In [20]:
%%sql
postgresql:///groceries

with dairy_sale19 as 
(select cast(p2.month as integer), count(p2.purchase_id) as "total_dsales"
from purchases_2019 p2
inner join categories c on (p2.purchase_id = c.purchase_id)
where c.category in ('whole milk', 'yogurt', 'domestic eggs')
group by month
order by month),

total_sale19 as 
(select cast(p2.month as integer), count(p2.purchase_id) as "total_sales"
from purchases_2019 p2
inner join categories c on (p2.purchase_id = c.purchase_id)
group by month
order by month),

dairy_sale20 as 
(select cast(left((right(p2.fulldate,5)),2) as integer) as month, count(p2.purchaseid) as "total_dsales"
from purchases_2020 p2
inner join categories c on (p2.purchaseid = c.purchase_id)
where c.category in ('whole milk', 'yogurt', 'domestic eggs')
group by month
order by month),

total_sale20 as 
(select cast(left((right(p2.fulldate,5)),2) as integer) as month, count(p2.purchaseid) as "total_sales"
from purchases_2020 p2
inner join categories c on (p2.purchaseid = c.purchase_id)
group by month
order by month)


select 
c.month, 
c.total_dsales as total_sales, 
round(((c.total_dsales * 100.0) /  d.total_sales),2) as "market_share", 
round((((c.total_dsales-a.total_dsales)*100.0)/a.total_dsales),2) as "year_change" 
from dairy_sale19 a
inner join total_sale19 b on a.month=b.month
inner join dairy_sale20 c on b.month=c.month
inner join total_sale20 d on c.month=d.month



12 rows affected.


month,total_sales,market_share,year_change
1,228,12.47,48.05
2,165,11.11,5.10
3,216,12.05,27.81
4,218,13.09,26.01
5,195,11.40,17.47
6,196,10.94,20.99
7,195,11.85,-0.51
8,231,11.77,32.00
9,180,11.16,4.65
10,193,11.61,17.68
